# Machine Learning: Project 2
## Task 2

First, the needed imports.
1. Pandas for reading the dataset
2. Numpy for some light vector operations
3. Sklearn:
    1. svm for the SVC module
    2. pca for dimentionality reduction based on feature covariance
4. Matplotlib for visualization

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

Read the datasets

In [7]:
X_train = pd.read_csv("./datasets/task2/14cancer_xtrain.csv", header=None)
Y_train = pd.read_csv("./datasets/task2/14cancer_ytrain.csv", header=None)

X_test = pd.read_csv("./datasets/task2/14cancer_xtest.csv", header=None)
Y_test = pd.read_csv("./datasets/task2/14cancer_ytest.csv", header=None)

The datasets are transposed, so transpose them back!

In [8]:
X_train = X_train.transpose()
Y_train = np.ravel(Y_train.transpose())

X_test = X_test.transpose()
Y_test = np.ravel(Y_test.transpose())

Get the mean and standard deviation of the sets and normalize

In [9]:
X = pd.concat([X_train, X_test])
mean = X.mean()
stdv = X.std()

# X_train = (X_train - mean)/stdv
# X_test = (X_test - mean)/stdv

Train and fit SVM classifiers

In [16]:
arr = []
for n_components in range(90, 126, 5):
    pca = PCA(n_components=n_components, svd_solver='full', whiten=True).fit(X_train)
    eigenvectors = pca.components_.reshape(n_components, X_train.shape[1])

    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)

    param_grid = {'C': [1, 1e1, 1e2, 1e3, 1e5, 1e6]}
    clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced', gamma='scale'), param_grid, cv=5, iid=False)
    clf = clf.fit(X_train_pca, Y_train)
    
    arr.append(clf.score(X_test_pca, Y_test))

In [18]:
# TODO: PLOT GRAPH